# Push Shift Data Collection

In [24]:
#The tools we need to pull Data from reddit

import pandas as pd
import numpy as np
import datetime as dt
import time 
import requests
import json

In [25]:
#Use the github API information and create the url. This is to get submissions

url = 'https://api.pushshift.io/reddit/search/submission/?subreddit=asktransgender'

In [26]:
# Next step is to do our requests to the reddit server

res = requests.get(url)
res


<Response [200]>

In [27]:
#We first used res.content and it gave us a mess, so now we have to convert
# This is a JSON object so now we need to convert it using the import JSON
# Now grabbing the data as it became a list. Will need to use slicing to pull things from the data
#Find out the length to give you a number of objects, 25 is the default, you can request more

len(res.json()['data'])

25

In [7]:
#Now we will grab the 1st thing as an example
# Everything is this list is now searchable to us
#using .keys() at the end to find the title or we can do res.json()['data'][0]['title] to find it
# We can use this method to find the 'selftext' as well

res.json()['data'][0]['title']

'Hello tranny bums!'

In [28]:
#This asktransgender subreddit does not have any self text in it

res.json()['data'][0]['selftext']

"When I look at a beautiful woman I see a beautiful woman when I look at a transitioning woman I see a beautiful woman I don't care she has a vagina or a penis she's absolutely beautiful"

In [29]:
res.json()['data'][0]['title']

"What's the difference between transitioning woman or just a woman?"

In [30]:
#How to go through and grab all 25 pieces of this data
#You do this by creating a for loop to get the list and dictionary to turn it into a data frame

posts = []

for post in  res.json()['data']:
    post_dict = {}
    post_dict['title'] = post['title']
    post_dict['text'] = post['selftext']
    post['auth'] = post['author']
    post_dict['time'] = post['created_utc']
    posts.append(post_dict)

df = pd.DataFrame(posts)

# In this search parameter where basically you want to find the lowest epic time in your data set 
# and then add it to your before search query. So you dont run into duplicates


In [31]:
#This helps you see where you should get the next data set so you dont have any duplicates
#Now try to combine everything into a huge function
#This is the earliest post in our data set and now to make sure we grab post that have been before
#so we are working back in time

df['time'].min()

1643258850

In [47]:
#This gives you the current epoch time
time.time()

1643281227.804953

# Push Shift Data Code

In [43]:
#You can also add comments into the function as well
#Function to pull data Make sure to pull current time always so you dont get repeats
#Yoou can also change the size from 25 to 100 by using the  '&size=100' at the url

def pushshift_query(subreddit='asktransgender', kind='submission', num_loops=10, current_time = 1643281227): #Grab from epoch converter
    #current_time()
    posts = []
    for query in range(num_loops):
        url = f'https://api.pushshift.io/reddit/search/{kind}/?subreddit={subreddit}&before={current_time}&size=100'
        res = requests.get(url)
        print(res.status_code)
        try:
            data = res.json()['data']
            for post in data:
                    post_dict = {}
                    post_dict['title'] = post['title']
                    post_dict['auth'] = post['author']
                    post_dict['time'] = post['created_utc']
                    post_dict['subreddit'] = subreddit
                    try:
                        post_dict['text'] = post['selftext']
                    except:
                        print('None')
                    
                    posts.append(post_dict)
                    current_time = pd.DataFrame(posts)['time'].min() 

        except:
            print('json failed')
        print(f'Current data frame has {len(posts)}rows')
           
        time.sleep(3) 

    return pd.DataFrame(posts)
                





# Ask Transgender Pulls 

In [48]:
asktransgender = pushshift_query(subreddit='asktransgender', kind='submission', num_loops=70, current_time = 1643281227) #overwrite the code in the time here

200
Current data frame has 100rows
200
Current data frame has 200rows
200
Current data frame has 300rows
200
Current data frame has 400rows
200
Current data frame has 500rows
200
Current data frame has 600rows
200
Current data frame has 700rows
200
None
Current data frame has 800rows
200
Current data frame has 900rows
200
Current data frame has 1000rows
200
Current data frame has 1100rows
200
Current data frame has 1200rows
200
Current data frame has 1300rows
200
Current data frame has 1400rows
200
Current data frame has 1500rows
200
Current data frame has 1600rows
200
Current data frame has 1700rows
200
Current data frame has 1800rows
200
Current data frame has 1900rows
200
None
Current data frame has 2000rows
200
Current data frame has 2100rows
200
Current data frame has 2200rows
200
Current data frame has 2300rows
200
None
Current data frame has 2400rows
200
Current data frame has 2500rows
200
Current data frame has 2600rows
200
Current data frame has 2700rows
200
Current data frame

In [49]:
asktransgender.duplicated().sum()

5

In [52]:
asktransgender.drop_duplicates(inplace=True)

In [53]:
asktransgender.duplicated().sum()

0

In [54]:
#Converting the pulls into a new dataframe

asktransgender.to_csv('ask_trans.csv')

# Rainbow Pulls 

In [55]:
rainbow = pushshift_query(subreddit='ainbow', kind='submission', num_loops=70, current_time = 1643281227) #/r/ainbow/

200
Current data frame has 100rows
200
Current data frame has 200rows
200
Current data frame has 300rows
200
Current data frame has 400rows
200
Current data frame has 500rows
200
Current data frame has 600rows
200
Current data frame has 700rows
200
Current data frame has 800rows
200
Current data frame has 900rows
200
None
Current data frame has 1000rows
200
Current data frame has 1100rows
200
Current data frame has 1200rows
200
Current data frame has 1300rows
200
Current data frame has 1400rows
200
Current data frame has 1499rows
200
Current data frame has 1599rows
200
Current data frame has 1699rows
200
Current data frame has 1799rows
200
Current data frame has 1899rows
200
Current data frame has 1999rows
200
Current data frame has 2099rows
200
Current data frame has 2199rows
200
Current data frame has 2299rows
200
Current data frame has 2399rows
200
Current data frame has 2499rows
200
Current data frame has 2599rows
200
Current data frame has 2699rows
200
Current data frame has 2799r

In [57]:
#Double Check if there are duplicates within the data

rainbow.duplicated().sum()

1

In [58]:
#Make sure the duplicate changes are final

rainbow.drop_duplicates(inplace=True)

In [59]:
rainbow.duplicated().sum()

0

In [60]:
rainbow.to_csv('rainbow.csv')

# Dataframes Check

In [61]:
type(rainbow)

pandas.core.frame.DataFrame

In [62]:
type(asktransgender)

pandas.core.frame.DataFrame

In [41]:
asktransgender.head(30)

,title,auth,time,subreddit,text
0,What's the difference between transitioning wo...,SnooChocolates8273,1643278788,asktransgender,When I look at a beautiful woman I see a beaut...
1,Did you also have this?,HoldTheStocks2,1643278667,asktransgender,When I was like 10-14 I was searching all over...
2,How many times a week do you dilatate?,Kind_Lemon,1643276284,asktransgender,"Hello!\n\nI am seven years post-surgical MtF, ..."
3,Coming out to my mother (again),mwnahas,1643274789,asktransgender,I’ve already done it about 1.5 years ago. She ...
4,Foot hurts is it hrt,Anastasia69Sanchez,1643273928,asktransgender,I heard foot gets smaller . Idk if I'm getting...
5,I'm on 2mg n see changes,Anastasia69Sanchez,1643273317,asktransgender,I sent from pills to injections 2mg n back to ...
6,Different odor while taking hrt,Anastasia69Sanchez,1643271524,asktransgender,Has anybody noticed after a year on HRT u have...
7,I got feelings...,tawmcruuze,1643271243,asktransgender,I'm a 27 year old... idk what the fuck I am. I...
8,How exactly do you tell if you’re binary trans...,Lower_Reference910,1643269923,asktransgender,"Hi all, so I’m a pre everything 24yo amab who ..."
9,How long did your imposter syndrome last?,ladyswift13,1643269674,asktransgender,Obviously I know everyone is different but I’m...


In [42]:
rainbow.head(30)

,title,auth,time,subreddit,text
0,Sarah Kahle - The Art of Being Queer,Artofbeingqueer,1643108002,ainbow,
1,"This made me cry, I wish every father was like...",Firewithin13,1643099363,ainbow,
2,"Why do I as a bi woman, wish I was a gay man?",OldReaction1969,1643081031,ainbow,[removed]
3,Gay Afghan Girl Searching for Gay Afghan Husband,AnonAfg,1643073455,ainbow,[removed]
4,Urge to watch hypno videos?,docomo98,1643064823,ainbow,[removed]
5,PAID Research Study for Young Adults Living wi...,n4connectproject,1643061744,ainbow,
6,Minecraft,PlanetZooster,1643060336,ainbow,[removed]
7,Minecraft,PlanetZooster,1643060260,ainbow,[removed]
8,Why is Asexual considered LGBT+?,throwreallysorryaway,1643059574,ainbow,[removed]
9,"I have to present a Ted talk, but I have to no...",fryingpansexual0405,1643055449,ainbow,


# Merge Data Sets 

In [ ]:
Whole_df = pd.concat([asktransgender, rainbow])
Whole_df

,title,auth,time,subreddit,text
0,Is there a health-guide or checklist for peopl...,20220125,1643111881,asktransgender,Background: My girlfriend (mtf) lives in 3rd w...
1,What trans sub-movement is going to far?,ManMadeStructure,1643110688,asktransgender,Many people confuse the broader community with...
2,Gender gp experiences,Ftm_wayne,1643109101,asktransgender,What’s everyone’s experience with gendergp lik...
3,Dysphoria has increased significantly,gfstuco,1643108294,asktransgender,"First off, I’ll be 19 in two months. I first w..."
4,"{NSFW} For in-tact ladies, are groin injuries ...",stompbixby,1643108208,asktransgender,[removed]
...,...,...,...,...,...
995,Am I the bad guy for cutting people off after ...,Substantial_Ride9635,1634135532,ainbow,[removed]
996,LGBTQIA+ Mindfulness Session Wednesday 7.00pm ...,beckywiththegdhair,1634119148,ainbow,
997,I'm scared that I'm only a demiboy because of ...,angydeku,1634088691,ainbow,[removed]
998,I wonder why i hear differently? 🤔,MotorArmadillo6917,1634087461,ainbow,


# Duplicate Check Code

In [ ]:
#data <-- Greyed out to search for duplicates witin data
#Since the word data is a data frame all you have to do is just attach '.duplicated().sum()'
#Now check for duplicates within that data that has been pulled
#Previous pull had 225 because of the url in the wrong position
#Now we pulled again and have zero 0 duplicates

data.duplicated().sum() #<-- This states code to check for duplicates



0

In [ ]:
#This is how you delete duplicates across the whole data frame,
#Leave inplace=True to keep the changes

any_df.drop_duplicates(inplace=True)

# Pretesting Code

In [ ]:
Whole_df['is_asktransgender'] = [1 if i == 'asktransgender' else 0 for i in Whole_df['subreddit']]


In [ ]:
Whole_df

,title,auth,time,subreddit,text,is_asktransgender
0,Is there a health-guide or checklist for peopl...,20220125,1643111881,asktransgender,Background: My girlfriend (mtf) lives in 3rd w...,1
1,What trans sub-movement is going to far?,ManMadeStructure,1643110688,asktransgender,Many people confuse the broader community with...,1
2,Gender gp experiences,Ftm_wayne,1643109101,asktransgender,What’s everyone’s experience with gendergp lik...,1
3,Dysphoria has increased significantly,gfstuco,1643108294,asktransgender,"First off, I’ll be 19 in two months. I first w...",1
4,"{NSFW} For in-tact ladies, are groin injuries ...",stompbixby,1643108208,asktransgender,[removed],1
...,...,...,...,...,...,...
995,Am I the bad guy for cutting people off after ...,Substantial_Ride9635,1634135532,ainbow,[removed],0
996,LGBTQIA+ Mindfulness Session Wednesday 7.00pm ...,beckywiththegdhair,1634119148,ainbow,,0
997,I'm scared that I'm only a demiboy because of ...,angydeku,1634088691,ainbow,[removed],0
998,I wonder why i hear differently? 🤔,MotorArmadillo6917,1634087461,ainbow,,0
